<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/evaluation_v5_precisionatK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.8.0 --quiet
!pip install transformers==4.15.0 --quiet
!pip install faiss-gpu transformers torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 58.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from transformers import BertTokenizer, TFBertModel

In [2]:
from google.colab import drive

# mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from tensorflow.keras.models import load_model
custom_objects = {'TFBertModel': TFBertModel}
model_v5data = load_model('./drive/MyDrive/two_tower_bert_v5.h5', custom_objects=custom_objects)

In [5]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
import numpy as np
# cls extraction
def generate_embedding_cls(text, model):
  bert_embedding_model = Model(inputs=[model.input],
                               outputs=model.get_layer('tf.__operators__.getitem').output)
  bert_train_tokenized = bert_tokenizer(text,
                                        max_length=128,
                                        truncation=True,
                                        padding='max_length',
                                        return_tensors='tf')
  bert_train_inputs = [bert_train_tokenized.input_ids,
                       bert_train_tokenized.token_type_ids,
                       bert_train_tokenized.attention_mask]
  return bert_embedding_model.predict(bert_train_inputs)


def embedding_vectordatabase(embedding):
  embeddings = np.array(embedding, dtype="float32")
  embeddings = embeddings.reshape(-1, embeddings.shape[-1])
  return embeddings

In [7]:
# create vector databases
import faiss
import numpy as np

dimension = 768  # Dimension of BERT-base embeddings
index_cls_v5 = faiss.IndexFlatL2(dimension)

In [8]:
level_four_job = ["Competitive salary for iPhone project management roles",
                  "Relocation benefits for Apple Cupertino campus positions",
                  "Exclusive health and wellness benefits for iPhone development team members",
                  "Stock options and bonuses for iPhone project milestones",
                  "Collaborative workspaces at Apple's Silicon Valley locations",
                  "Professional development programs for iOS project leads",
                  "Confidentiality agreements for upcoming iPhone projects",
                  "Global travel opportunities for iPhone supplier audits",
                  "Flexible working hours for Apple's product development teams",
                  "Team-building retreats for iPhone project managers"]
for input in level_four_job:
  embeddings = generate_embedding_cls(input, model_v5data)
  index_cls_v5.add(embedding_vectordatabase(embeddings))
print(index_cls_v5.ntotal)

10


In [9]:
df_GPT_evaluation =  pd.read_csv("./drive/MyDrive/job_related_GPT_evaluation_updated.csv", error_bad_lines=False)
df_GPT_evaluation

<ipython-input-9-5fb7bff6d40a>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_GPT_evaluation =  pd.read_csv("./drive/MyDrive/job_related_GPT_evaluation_updated.csv", error_bad_lines=False)


,prompt,job_related_info_index,top_3_relevant_blacklist_items
0,"Recently, I've noticed better sleep after expl...",7,"7,2,0"
1,"Recently, I've noticed higher energy levels af...",2,"2,3,0"
2,"Recently, I've noticed less anxiety after taki...",5,"5,0,2"
3,"Recently, I've noticed increased creativity af...",7,"7,0,2"
4,"Recently, I've noticed more patience after par...",2,"2,3,5"
...,...,...,...
295,Since immersing in flexible working hours for ...,8,"8,2,5"
296,Since exploring the advantages of confidential...,6,"6,0,2"
297,Since utilizing confidentiality agreements for...,6,"6,0,2"
298,Since utilizing exclusive health and wellness ...,2,"2,3,5"


In [10]:
k = 3
indices_ss = []
distance_ss = []
for prompt in df_GPT_evaluation['prompt'][150:300]:
    embeddings_v5 = generate_embedding_cls(prompt, model_v5data)
    distances_v5, indices_v5 = index_cls_v5.search(embeddings_v5, k)
    indices_ss.append(indices_v5)
    distance_ss.append(distance_ss)

In [33]:
indices_ss_series = pd.Series(indices_ss)
distance_ss_series = pd.Series(distance_ss)
df_GPT_evaluation_sechalf = df_GPT_evaluation.iloc[150:300]
df_GPT_evaluation_sechalf.reset_index(drop=True, inplace=True)
df_GPT_evaluation_sechalf['indices_ss'] = indices_ss_series
df_GPT_evaluation_sechalf['distance_ss'] = distance_ss_series

<ipython-input-33-1d8d0aabae40>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GPT_evaluation_sechalf['indices_ss'] = indices_ss_series
<ipython-input-33-1d8d0aabae40>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GPT_evaluation_sechalf['distance_ss'] = distance_ss_series


In [37]:
df_GPT_evaluation_sechalf.to_csv('./drive/MyDrive/job_related_GPT_evaluation_v5_ss_sechalf_2.csv', index=False)

Evaluation

In [17]:
# precision and recall at K
# if top 3 contains the correct one?

df_first_half =  pd.read_csv("./drive/MyDrive/job_related_GPT_evaluation_v5_ss_sechalf_2.csv", error_bad_lines=False)
df_second_half =  pd.read_csv("./drive/MyDrive/job_related_GPT_evaluation_v5_ss.csv", error_bad_lines=False)
df_GPT_evaluation = pd.concat([df_first_half, df_second_half])
df_GPT_evaluation

<ipython-input-17-90b7056af06b>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_first_half =  pd.read_csv("./drive/MyDrive/job_related_GPT_evaluation_v5_ss_sechalf_2.csv", error_bad_lines=False)
<ipython-input-17-90b7056af06b>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_second_half =  pd.read_csv("./drive/MyDrive/job_related_GPT_evaluation_v5_ss.csv", error_bad_lines=False)


,prompt,job_related_info_index,top_3_relevant_blacklist_items,indices_ss,distance_ss
0,"Recently, I've noticed better sleep after expl...",7,"7,2,0",[[3 7 6]],"[[...], [...], [...], [...], [...], [...], [....."
1,"Recently, I've noticed higher energy levels af...",2,"2,3,0",[[1 9 2]],"[[...], [...], [...], [...], [...], [...], [....."
2,"Recently, I've noticed less anxiety after taki...",5,"5,0,2",[[1 4 9]],"[[...], [...], [...], [...], [...], [...], [....."
3,"Recently, I've noticed increased creativity af...",7,"7,0,2",[[4 8 1]],"[[...], [...], [...], [...], [...], [...], [....."
4,"Recently, I've noticed more patience after par...",2,"2,3,5",[[4 1 8]],"[[...], [...], [...], [...], [...], [...], [....."
...,...,...,...,...,...
145,The decision to focus on team-building retreat...,9,"9,0,3",[[9 2 1]],"[[...], [...], [...], [...], [...], [...], [....."
146,Since making the most out of global travel opp...,7,"7,0,2",[[7 6 8]],"[[...], [...], [...], [...], [...], [...], [....."
147,Engaging with professional development program...,5,"5,0,2",[[4 1 9]],"[[...], [...], [...], [...], [...], [...], [....."
148,The decision to focus on relocation benefits f...,1,"1,2,0",[[1 4 9]],"[[...], [...], [...], [...], [...], [...], [....."


In [19]:
df_GPT_evaluation['indices_ss'] = df_GPT_evaluation['indices_ss'].apply(lambda x: ','.join(map(str, np.fromstring(x.strip('[]'), dtype=int, sep=' '))))

In [20]:
df_GPT_evaluation

,prompt,job_related_info_index,top_3_relevant_blacklist_items,indices_ss,distance_ss
0,"Recently, I've noticed better sleep after expl...",7,"7,2,0","3,7,6","[[...], [...], [...], [...], [...], [...], [....."
1,"Recently, I've noticed higher energy levels af...",2,"2,3,0","1,9,2","[[...], [...], [...], [...], [...], [...], [....."
2,"Recently, I've noticed less anxiety after taki...",5,"5,0,2","1,4,9","[[...], [...], [...], [...], [...], [...], [....."
3,"Recently, I've noticed increased creativity af...",7,"7,0,2","4,8,1","[[...], [...], [...], [...], [...], [...], [....."
4,"Recently, I've noticed more patience after par...",2,"2,3,5","4,1,8","[[...], [...], [...], [...], [...], [...], [....."
...,...,...,...,...,...
145,The decision to focus on team-building retreat...,9,"9,0,3","9,2,1","[[...], [...], [...], [...], [...], [...], [....."
146,Since making the most out of global travel opp...,7,"7,0,2","7,6,8","[[...], [...], [...], [...], [...], [...], [....."
147,Engaging with professional development program...,5,"5,0,2","4,1,9","[[...], [...], [...], [...], [...], [...], [....."
148,The decision to focus on relocation benefits f...,1,"1,2,0","1,4,9","[[...], [...], [...], [...], [...], [...], [....."


In [22]:
def calculate_precision_recall_at_k(relevant_items, recommended_items, k=3):
    relevant_set = set(relevant_items.split(','))
    recommended_set = set(recommended_items.split(','))
    relevant_and_recommended = recommended_set.intersection(relevant_set)
    precision = len(relevant_and_recommended) / k
    recall = len(relevant_and_recommended) / len(relevant_set)
    return precision, recall

df_GPT_evaluation['precision@3'], df_GPT_evaluation['recall@3'] = zip(*df_GPT_evaluation.apply(lambda x: calculate_precision_recall_at_k(x['top_3_relevant_blacklist_items'], x['indices_ss']), axis=1))

In [25]:
def check_correctness(relevant_items, recommended_items, job_related_info_index):
    relevant_correct = str(job_related_info_index) in relevant_items.split(',')
    recommended_correct = str(job_related_info_index) in recommended_items.split(',')
    return relevant_correct, recommended_correct

df_GPT_evaluation['GPT_correct'], df_GPT_evaluation['twotower_correct'] = zip(*df_GPT_evaluation.apply(lambda x: check_correctness(x['top_3_relevant_blacklist_items'], x['indices_ss'], x['job_related_info_index']), axis=1))

In [26]:
df_GPT_evaluation

,prompt,job_related_info_index,top_3_relevant_blacklist_items,indices_ss,distance_ss,precision@3,recall@3,GPT_correct,twotower_correct
0,"Recently, I've noticed better sleep after expl...",7,"7,2,0","3,7,6","[[...], [...], [...], [...], [...], [...], [.....",0.333333,0.333333,True,True
1,"Recently, I've noticed higher energy levels af...",2,"2,3,0","1,9,2","[[...], [...], [...], [...], [...], [...], [.....",0.333333,0.333333,True,True
2,"Recently, I've noticed less anxiety after taki...",5,"5,0,2","1,4,9","[[...], [...], [...], [...], [...], [...], [.....",0.000000,0.000000,True,False
3,"Recently, I've noticed increased creativity af...",7,"7,0,2","4,8,1","[[...], [...], [...], [...], [...], [...], [.....",0.000000,0.000000,True,False
4,"Recently, I've noticed more patience after par...",2,"2,3,5","4,1,8","[[...], [...], [...], [...], [...], [...], [.....",0.000000,0.000000,True,False
...,...,...,...,...,...,...,...,...,...
145,The decision to focus on team-building retreat...,9,"9,0,3","9,2,1","[[...], [...], [...], [...], [...], [...], [.....",0.333333,0.333333,True,True
146,Since making the most out of global travel opp...,7,"7,0,2","7,6,8","[[...], [...], [...], [...], [...], [...], [.....",0.333333,0.333333,True,True
147,Engaging with professional development program...,5,"5,0,2","4,1,9","[[...], [...], [...], [...], [...], [...], [.....",0.000000,0.000000,True,False
148,The decision to focus on relocation benefits f...,1,"1,2,0","1,4,9","[[...], [...], [...], [...], [...], [...], [.....",0.333333,0.333333,True,True


In [27]:
# accuracy by index:

accuracy_by_index = df_GPT_evaluation.groupby('job_related_info_index')[['GPT_correct', 'twotower_correct']].mean()
average_precision = df_GPT_evaluation.groupby('job_related_info_index')['precision@3'].mean()
average_recall = df_GPT_evaluation.groupby('job_related_info_index')['recall@3'].mean()

print(accuracy_by_index, average_precision, average_recall)

                        GPT_correct  twotower_correct
job_related_info_index                               
0                               1.0          0.000000
1                               1.0          0.846154
2                               1.0          0.800000
3                               1.0          0.750000
4                               1.0          0.346154
5                               1.0          0.025000
6                               1.0          0.714286
7                               1.0          0.571429
8                               1.0          0.000000
9                               1.0          0.619048 job_related_info_index
0    0.393939
1    0.397436
2    0.411111
3    0.361111
4    0.307692
5    0.283333
6    0.333333
7    0.333333
8    0.318182
9    0.373016
Name: precision@3, dtype: float64 job_related_info_index
0    0.393939
1    0.397436
2    0.411111
3    0.361111
4    0.307692
5    0.283333
6    0.333333
7    0.333333
8    0.318182
9    0

In [ ]:
# bad 0, 4, 5, 8

level_four_job = ["Competitive salary for iPhone project management roles",
                  "Relocation benefits for Apple Cupertino campus positions",
                  "Exclusive health and wellness benefits for iPhone development team members",
                  "Stock options and bonuses for iPhone project milestones",
                  "Collaborative workspaces at Apple's Silicon Valley locations",
                  "Professional development programs for iOS project leads",
                  "Confidentiality agreements for upcoming iPhone projects",
                  "Global travel opportunities for iPhone supplier audits",
                  "Flexible working hours for Apple's product development teams",
                  "Team-building retreats for iPhone project managers"]

In [39]:
df_finetune = df_GPT_evaluation[df_GPT_evaluation['job_related_info_index'].isin([0, 4, 5, 8])]['prompt']
df_finetune['related_to_job'] = 1
df_finetune.to_csv('./drive/MyDrive/df_finetune.csv', index=False)

<ipython-input-39-4044fe5b7ef6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_finetune['related_to_job'] = 1
